In [356]:
import pandas as pd

In [357]:
import time

start_time = time.time()

In [358]:
def cabecera_resumen_mot(row, format):
    worksheet.write('A' + str(row), 'Categoría', format)
    worksheet.write('B' + str(row), 'Cadena', format)
    worksheet.write('C' + str(row), 'Si', format)
    worksheet.write('D' + str(row), 'Si pero el precio no corresponde', format)
    worksheet.write('E' + str(row), 'No ha caido la promoción', format)
    worksheet.write('F' + str(row), 'No puedo señalizar', format)
    worksheet.write('G' + str(row), 'No tengo suficiente producto', format)
    worksheet.write('H' + str(row), 'No El gerente no me permite ejecutar', format)
    worksheet.write('I' + str(row), 'No he ejecutado', format)
    worksheet.write('J' + str(row), 'Ejecutado', format)
    worksheet.write('K' + str(row), 'NO Ejecutado', format)
    worksheet.write('L' + str(row), '% Ejecución', format)
    worksheet.write('M' + str(row), '% No ha caido la promoción', format)
    worksheet.write('N' + str(row), '% No puedo señalizar', format)
    worksheet.write('O' + str(row), '% No tengo suficiente producto', format)
    worksheet.write('P' + str(row), '% No El gerente no me permite ejecutar', format)
    worksheet.write('Q' + str(row), '% No he ejecutado', format)

In [359]:
stores_file = 'C:\\Users\\gonzalezcoronado.o\\PycharmProjects\\Promos\\stores_clean.xlsx'

In [360]:
stores = pd.read_excel(stores_file, 0)

In [361]:
belleza = pd.read_excel('C:\\Users\\gonzalezcoronado.o\\PycharmProjects\\Promos\\Plan Promocional Belleza Mayo 2017.xlsx', 4, 2)

In [362]:
preguntas = belleza[['Categoría', 'NOMBRE DE LA INICIATIVA']]

In [363]:
iniciativa_belleza = pd.read_csv('C:\\Users\\gonzalezcoronado.o\\PycharmProjects\\Promos\\lecturas_iniciativa_11.csv', encoding='mbcs')

In [364]:
writter = pd.ExcelWriter('C:\\Users\\gonzalezcoronado.o\\PycharmProjects\\Promos\\RepPromoBelleza.xlsx')

In [365]:
iniciativas = iniciativa_belleza[
    ['Cadena', 'Región', 'Zona', 'Tienda ID', 'Nombre Tienda', 'Fecha Captura', 'Grupo Categorias',
     'Iniciativa', 'Apoyo', 'Respuesta Opc.Multiple/Texto Abierto']]

In [366]:
respuestas = iniciativas['Respuesta Opc.Multiple/Texto Abierto'].replace('1.Si\r\n2.No', 'Si')
respuestas = respuestas.replace('he ejecutado\r\n6.No', 'No he ejecutado')
respuestas = respuestas.replace('he ejecutado\n6.No', 'No he ejecutado')
respuestas = respuestas.replace('he ejecutado', 'No he ejecutado')
respuestas = respuestas.replace('no tengo suficiente producto\r\n3.No', 'No tengo suficiente producto')
respuestas = respuestas.replace('no tengo suficiente producto\n3.No', 'No tengo suficiente producto')
respuestas = respuestas.replace('no ha caido la promoción\r\n4.No', 'No ha caido la promoción')
respuestas = respuestas.replace('no ha caido la promoción\n4.No', 'No ha caido la promoción')
respuestas = respuestas.replace('1.Si\r\n2.Si', 'Si')
respuestas = respuestas.replace('1.Si\n2.Si', 'Si')
respuestas = respuestas.replace('1.Si\n2.No', 'Si')
respuestas = respuestas.replace('pero el precio no corresponde.  \r\n3.No', 'Si pero el precio no corresponde')
respuestas = respuestas.replace('pero el precio no corresponde.  \n3.No', 'Si pero el precio no corresponde')
respuestas = respuestas.replace('no ha caido la promoción\r\n5.No', 'No ha caido la promoción')
respuestas = respuestas.replace('no ha caido la promoción\n5.No', 'No ha caido la promoción')
respuestas = respuestas.replace('no tengo suficiente producto\r\n4.No', 'No tengo suficiente producto')
respuestas = respuestas.replace('no tengo suficiente producto\n4.No', 'No tengo suficiente producto')
respuestas = respuestas.replace('no puedo señalizar\r\n5.No', 'No puedo señalizar')
respuestas = respuestas.replace('no puedo señalizar\r\n6.No', 'No puedo señalizar')

In [367]:
iniciativas = iniciativas.join(respuestas, rsuffix='_respuestas')

In [368]:
preguntas_id = iniciativas['Apoyo'].str.extract("(?P<PreguntaID>\w{5}\d{1,3})(?P<Pregunta>¿.+)", expand=False)

In [369]:
iniciativas = iniciativas.join(preguntas_id)

In [370]:
preguntas = preguntas.set_index('NOMBRE DE LA INICIATIVA')

In [371]:
iniciativas = iniciativas.join(preguntas, on='Pregunta')

In [372]:
stores = stores[['Stores ID', '# Sucursal Cliente', 'Cadena', 'Formato', 'Nombre Tienda', 'Estatus de Tienda', 'Canal']]

In [373]:
stores = stores.drop_duplicates()

In [374]:
iniciativas = iniciativas.join(stores.set_index('Stores ID'), on='Tienda ID', rsuffix='_stores')

In [375]:
columnas_iniciativas = iniciativas.columns

In [376]:
iniciativas = iniciativas.sort_values(by='Fecha Captura')

In [377]:
iniciativas = iniciativas.drop_duplicates(['Nombre Tienda', 'Pregunta'], keep='last').values

In [378]:
iniciativas = pd.DataFrame(iniciativas, columns=columnas_iniciativas)

In [379]:
data = iniciativas[['Canal', 'Categoría', 'Cadena', 'Región', 'Zona', 'Tienda ID', 'Nombre Tienda', 'Fecha Captura',
                    'Iniciativa', 'Pregunta', 'Respuesta Opc.Multiple/Texto Abierto_respuestas']]

In [380]:
order = data.groupby(['Canal', 'Categoría', 'Cadena', 'Pregunta', 'Respuesta Opc.Multiple/Texto Abierto_respuestas'],
                     as_index=False).count()

In [381]:
order = order[['Canal', 'Categoría', 'Cadena', 'Pregunta', 'Respuesta Opc.Multiple/Texto Abierto_respuestas', 'Zona']]

In [382]:
categorias = order.pivot_table(index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
                               columns='Respuesta Opc.Multiple/Texto Abierto_respuestas', values='Zona', fill_value=0).reset_index(
    ['Canal', 'Categoría', 'Cadena', 'Pregunta'])

In [383]:
categorias = categorias.assign(Ejecutado=(categorias['Si'] + categorias['Si pero el precio no corresponde']))

In [384]:
categorias = categorias.assign(No_Ejecutado=(
    categorias['No ha caido la promoción'] + categorias['No he ejecutado'] + categorias['No puedo señalizar'] +
    categorias['No tengo suficiente producto']))

In [385]:
categorias = categorias.assign(
    Porcentaje_Ejecución=(categorias['Ejecutado'] / (categorias['Ejecutado'] + categorias['No_Ejecutado'])))

In [386]:
categorias = categorias.rename(columns={'No_Ejecutado': 'No Ejecutado', 'Porcentaje_Ejecución': '% Ejecución'})

In [387]:
categorias = categorias[['Canal', 'Categoría', 'Cadena', 'Pregunta', 'No El gerente no me permite ejecutar', 
                         'No ha caido la promoción', 'No he ejecutado', 'No puedo señalizar', 'No tengo suficiente producto', 
                         'Si', 'Si pero el precio no corresponde', 'Ejecutado', 'No Ejecutado', '% Ejecución']]

In [388]:
resumen_gos = data.groupby(['Respuesta Opc.Multiple/Texto Abierto_respuestas', 'Categoría'], as_index=False).count()
resumen_gos = resumen_gos.pivot_table(index=['Respuesta Opc.Multiple/Texto Abierto_respuestas'], columns='Categoría',
                                      values='Zona', fill_value=0)

In [389]:
sum_row = resumen_gos.sum()
sum_row = pd.DataFrame(sum_row).T

In [390]:
resumen_gos = resumen_gos.append(sum_row)

In [391]:
resumen_gos['% Cuidado Femenino'] = (resumen_gos['Cuidado Femenino'] / resumen_gos.iloc[7,0])
resumen_gos['% Cuidado Oral'] = (resumen_gos['Cuidado Oral'] / resumen_gos.iloc[7,1])
resumen_gos['% Cuidado del Cabello'] = (resumen_gos['Cuidado del Cabello'] / resumen_gos.iloc[7,2])
resumen_gos['% Desodorantes'] = (resumen_gos['Desodorantes'] / resumen_gos.iloc[7,3])
resumen_gos['% Hojas y máquinas'] = (resumen_gos['Hojas y Máquinas'] / resumen_gos.iloc[7,4])
resumen_gos = resumen_gos.iloc[0:7,0:10]
resumen_gos.to_excel(writter, 'Resumen GOs')

In [392]:
active_row_names = ['No El gerente no me permite ejecutar', 'No ha caido la promoción', 'No tengo suficiente producto', 
                    'No puedo señalizar']
active_filter = [row in active_row_names for row in resumen_gos.index]
inactive_filter = [not row for row in active_filter]
active = resumen_gos.loc[active_filter].sum()
active  = pd.DataFrame(active).transpose()
active.index=["NO, causal CT"] 
segundo = resumen_gos.append(active, ignore_index=False)

In [393]:
active_row_names = ['Si', 'Si pero el precio no corresponde']
active_filter = [row in active_row_names for row in segundo.index]
inactive_filter = [not row for row in active_filter]
active = resumen_gos.loc[active_filter].sum()
active  = pd.DataFrame(active).transpose()
active.index=["SI"] 
segundo = segundo.loc[inactive_filter].append(active, ignore_index=False)

In [394]:
segundo = segundo.sort_index(1)

In [395]:
segundo = segundo[['Cuidado Femenino', 'Cuidado Oral', 'Cuidado del Cabello', 'Desodorantes', 'Hojas y Máquinas',  
                   '% Cuidado Femenino', '% Cuidado Oral', '% Cuidado del Cabello', '% Desodorantes', '% Hojas y máquinas']]

In [396]:
segundo.to_excel(writter, 'Resumen GOs', startrow=11)

In [397]:
no_ejecutado_region = data.groupby(['Región', 'Categoría','Respuesta Opc.Multiple/Texto Abierto_respuestas'],
                     as_index=False).count()

In [398]:
no_ejecutado_region = no_ejecutado_region[no_ejecutado_region['Respuesta Opc.Multiple/Texto Abierto_respuestas'] == 
                                          'No he ejecutado']

In [399]:
no_ejecutado_region = no_ejecutado_region.pivot_table(index='Región', columns='Categoría', values='Canal', fill_value=0)

In [400]:
no_ejecutado_region.to_excel(writter, 'Resumen GOs', startrow=22)

In [401]:
reporte_mot = data.groupby(['Categoría', 'Cadena', 'Respuesta Opc.Multiple/Texto Abierto_respuestas'], as_index=False).count()

In [402]:
reporte_mot = reporte_mot.pivot_table(index=['Categoría', 'Cadena'], columns='Respuesta Opc.Multiple/Texto Abierto_respuestas',
                                      values='Zona', fill_value=0)

In [403]:
reporte_mot = reporte_mot.reset_index()

In [404]:
reporte_mot = reporte_mot[
    ['Categoría', 'Cadena', 'Si', 'Si pero el precio no corresponde', 'No ha caido la promoción', 'No puedo señalizar',
     'No tengo suficiente producto', 'No El gerente no me permite ejecutar', 'No he ejecutado']]

In [405]:
reporte_mot['Ejecutado'] = (reporte_mot['Si'] + reporte_mot['Si pero el precio no corresponde'])

In [406]:
reporte_mot['No Ejecutado'] = (reporte_mot['No ha caido la promoción'] + reporte_mot['No he ejecutado'] + 
                               reporte_mot['No puedo señalizar'] + reporte_mot['No tengo suficiente producto'] + 
                               reporte_mot['No El gerente no me permite ejecutar'])

In [407]:
reporte_mot['% Ejecución'] = (reporte_mot['Ejecutado'] / (reporte_mot['Ejecutado'] + reporte_mot['No Ejecutado']))

In [408]:
reporte_mot['% No ha caido la promoción'] = (reporte_mot['No ha caido la promoción'] / reporte_mot['No Ejecutado'])

In [409]:
reporte_mot['% No puedo señalizar'] = (reporte_mot['No puedo señalizar'] / reporte_mot['No Ejecutado'])

In [410]:
reporte_mot['% No tengo suficiente producto'] = (reporte_mot['No tengo suficiente producto'] / reporte_mot['No Ejecutado'])

In [411]:
reporte_mot['% No El gerente no me permite ejecutar'] = (reporte_mot['No El gerente no me permite ejecutar'] / 
                                                      reporte_mot['No Ejecutado'])

In [412]:
reporte_mot['% No he ejecutado'] = (reporte_mot['No he ejecutado'] / reporte_mot['No Ejecutado'])
reporte_mot = reporte_mot.fillna(0)

In [413]:
workbook = writter.book

In [414]:
categorias_values = categorias.Categoría.unique()

In [415]:
formato_cabecera = workbook.add_format()
formato_cabecera.set_font_color('white')
formato_cabecera.set_align('center')
formato_cabecera.set_bg_color('#222B35')
formato_cabecera.set_bold()
formato_cabecera.set_text_wrap()
formato_cabecera.set_border()

In [416]:
formato_titulos = formato_cabecera

In [417]:
row = 2
for cat in categorias_values:
    reporte_mot[reporte_mot['Categoría'] == cat].to_excel(writter, 'Resumen MOT', startrow=row + 1, index=False, header=False)
    worksheet = writter.sheets['Resumen MOT']
    valores = reporte_mot.Cadena[reporte_mot['Categoría'] == cat].count()
    worksheet.merge_range('M' + str(row - 1) + ':Q' + str(row - 1), 'Razones para no ejecutar', formato_cabecera)
    worksheet.merge_range('M' + str(row) + ':O' + str(row), 'Causal CT', formato_cabecera)
    worksheet.merge_range('P' + str(row) + ':Q' + str(row), 'Causal SDO', formato_cabecera)
    cabecera_resumen_mot(row + 1, formato_cabecera)
    worksheet.set_row(row, 33.75)
    worksheet.conditional_format('L' + str(row + 1) + ':L' + str(row + valores + 1), {'type': '3_color_scale'})
    row += (valores + 4)

In [418]:
for cat in categorias_values:
    categorias[categorias['Categoría'] == cat].to_excel(writter, cat, index=False)

In [419]:
worksheet = writter.sheets['Resumen MOT']

In [420]:
porcentaje = workbook.add_format({'num_format': '0%'})

In [421]:
worksheet.set_column(0, 0, 18.29)
worksheet.set_column(1, 1, 21.29)
worksheet.set_column('C:K', None, None, {'hidden': True})
worksheet.set_column('L:Q', 21, porcentaje)

0

In [422]:
order[order['Cadena'] == 'AC Soriana'].pivot_table(index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
                                                   columns='Respuesta Opc.Multiple/Texto Abierto_respuestas', values='Zona',
                                                   fill_value=0).reset_index().to_excel(writter, 'AC Soriana',
                                                                                        index=False)

In [423]:
order[order['Cadena'] == 'AD Chedraui'].pivot_table(index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
                                                    columns='Respuesta Opc.Multiple/Texto Abierto_respuestas', values='Zona',
                                                    fill_value=0).reset_index().to_excel(writter, 'AD Chedraui',
                                                                                         index=False)

In [424]:
order[order['Cadena'] == 'AE Comercial Mexicana'].pivot_table(index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
                                                              columns='Respuesta Opc.Multiple/Texto Abierto_respuestas',
                                                              values='Zona',
                                                              fill_value=0).reset_index().to_excel(writter,
                                                                                                   'AE Comercial Mexicana',
                                                                                                   index=False)

In [425]:
order[(order['Cadena'] == 'AB Bodega Aurrera') | (order['Cadena'] == 'AA Supercenter') | (order[
                                                                                              'Cadena'] == 'Superama')].pivot_table(
    index=['Canal', 'Categoría', 'Cadena', 'Pregunta'],
    columns='Respuesta Opc.Multiple/Texto Abierto_respuestas', values='Zona',
    fill_value=0).reset_index().to_excel(writter, 'WM',
                                         index=False)

In [426]:
workbook.close()
writter.save()

In [427]:
print('Ejecutado en: ' + str(time.time() - start_time) + ' segundos')

Ejecutado en: 3.865036725997925 segundos
